# Elastic engine demo

In [1]:
import datalabframework as dlf
from datalabframework import utils
dlf.project.info()

{'profile': 'default',
 'filename': 'elastic.ipynb',
 'rootpath': '/home/thuc/teko/product-search/dlf-demos_fork/demos/elastic',
 'workdir': '/home/thuc/teko/product-search/dlf-demos_fork/demos/elastic'}

In [3]:
metadata = dlf.params.metadata()
utils.pretty_print(metadata)

engines:
  spark-local:
    config:
      jobname: default
      master: local[*]
    context: spark
loggers:
  severity: info
  stream:
    enable: true
profile: default
providers:
  elastic_test:
    hostname: 123.31.32.226
    port: 9200
    service: elastic
  local_filesystem:
    format: csv
    path: data
    service: local
resources:
  .keywords:
    path: keywords.csv
    provider: local_filesystem
    read:
      mapping:
        abc:
          drop: true
          fields:
            keyword:
              ignore_above: 256
              type: keyword
          fill: value
          name: cds
          type: text
      options:
        header: true
        inferSchema: true
  .keywords_elk:
    path: search_keywords_dev2
    provider: elastic_test
    write:
      options:
        mappings:
          doc_type: keyword
          properties:
            api_result:
              type: double
            count:
              type: integer
            gg:
              type: inte

In [4]:
import datalabframework as dlf
engine = dlf.engines.get('spark-local')

PYSPARK_SUBMIT_ARGS:  pyspark-shell


In [5]:
res = engine.read(".keywords_search")
res.show()

2018-10-16 15:34:46,995 - INFO - f318bdd - datalabframework-demos.git - thuc - elastic.ipynb - engine.read - {'format': 'parquet', 'service': 'elastic', 'path': '/search_keywords_dev2/keyword/', 'url': 'http://123.31.32.226:9200//search_keywords_dev2/keyword/'}
+--------------------+--------------------+---------+-------+----------+-----+---+--------------+-----------------+-----------+--------+---+---------+-----------+--------------+
|                 _id|              _index|   _score|  _type|api_result|count| gg|gg_impressions|      gg_position|impressions|position| pv|pv_result|      query|query_wo_tones|
+--------------------+--------------------+---------+-------+----------+-----+---+--------------+-----------------+-----------+--------+---+---------+-----------+--------------+
|6Pf_e2YBknddiPsCp-8u|search_keywords_dev2|968.68555|keyword|    1452.0|    7|  1|      115890.0|4.356657846714286|    26945.0|     1.0|  1|    119.0|laptop dell|   laptop dell|
|WPf_e2YBknddiPsC4_q8|sear

In [6]:
dlf.project.repository()

{'type': 'git',
 'committer': 'natbusa',
 'hash': 'f318bdd',
 'commit': 'f318bdd1b54265bf04e35e91af50ad280b4776c3',
 'branch': 'master',
 'url': 'git@github.com:ncthuc/datalabframework-demos.git',
 'name': 'datalabframework-demos.git',
 'date': '2018-10-16T11:56:59+07:00',
 'clean': False}

In [7]:
type(res)

pyspark.sql.dataframe.DataFrame

In [8]:
df = engine.read(".keywords")

2018-10-16 15:34:58,828 - INFO - f318bdd - datalabframework-demos.git - thuc - elastic.ipynb - engine.read - {'format': 'csv', 'service': 'local', 'path': 'keywords.csv', 'url': 'file:///home/thuc/teko/product-search/dlf-demos_fork/demos/elastic/data/keywords.csv'}


In [9]:
df.show()

+--------------------+--------------------+-----------+--------+--------------+-----------+----------+---------+---+---+-----+
|      query_wo_tones|               query|impressions|position|gg_impressions|gg_position|api_result|pv_result| pv| gg|count|
+--------------------+--------------------+-----------+--------+--------------+-----------+----------+---------+---+---+-----+
|        asus h110m-d|        asus h110m-d|195.8090665|    null|         570.0|3.335087719|     458.0|      0.0|  0|  1|    1|
|           ddr3 1600|           ddr3 1600|      202.0|     1.0|          86.0|        2.5|      30.0|     36.0|  1|  1|    1|
|         epson l1300|         epson l1300|191.3432457|    null|         557.0|5.057450628|       1.0|      1.0|  0|  1|    1|
| intel pentium g4600| intel pentium g4600|166.2659442|    null|         484.0|3.423553719|       1.0|      0.0|  0|  1|    1|
|               j250f|               j250f|220.1993186|    null|         641.0|3.653666147|       1.0|      3.0

In [10]:
# import datalabframework as dlf
# from datalabframework import utils
# engine = dlf.engines.get('spark-local')
# df = engine.read(".keywords")

In [12]:
engine.write(df, ".keywords_elk", mode='overwrite')

{'index': {'mapping': {'total_fields': {'limit': 1024}}, 'number_of_replicas': 3, 'number_of_shards': 1}}
{'api_result': {'type': 'double'}, 'count': {'type': 'integer'}, 'gg': {'type': 'integer'}, 'gg_clicks': {'type': 'double'}, 'gg_impressions': {'type': 'double'}, 'gg_position': {'type': 'double'}, 'pv': {'type': 'integer'}, 'pv_impressions': {'type': 'double'}, 'pv_position': {'type': 'double'}, 'pv_result': {'type': 'double'}, 'query': {'fields': {'keyword': {'ignore_above': 128, 'type': 'keyword'}}, 'type': 'text'}, 'query_wo_tones': {'fields': {'keyword': {'ignore_above': 128, 'type': 'keyword'}}, 'type': 'text'}}
2018-10-16 15:35:32,562 - INFO - f318bdd - datalabframework-demos.git - thuc - elastic.ipynb - engine.write - {'format': 'parquet', 'service': 'elastic', 'path': 'search_keywords_dev2', 'url': 'http://123.31.32.226:9200/search_keywords_dev2'}
